In [1]:
# from dask.distributed import Client
# client = Client('127.0.0.1:35517')

In [1]:
import cartopy
import cmocean
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.colorbar
import xarray as xr
import numpy

In [3]:
# from dask.distributed import Client

In [4]:
# client = Client('140.208.147.173:34671')

In [5]:
# client

## Change files beeing read

In [2]:
%%time
#300yr OM4files, date 198801 - 200712

OM4p25 = xr.open_zarr('/work/Nkeh.Boh/SST/OM4p25_zarr/OM4p25.zarr')
# OM4p25z = xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/OM4p25z_tos_300yr.zarr')
# OM4p5nep = xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/OM4p5nep_tos_300yr.zarr')
# OM4p5mle =xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/OM4p5mle_tos_300yr.zarr')
OM4p5 = xr.open_zarr('/work/Nkeh.Boh/SST/OM4p5_zarr/OM4p5.zarr')

#WOA files
W25 = xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/WOA25.zarr')
W5 = xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/WOA05.zarr')

#Grid files
G25 = xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/Ocean_grid25.zarr')
G5 = xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/Ocean_grid05.zarr')
M25 = xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/Ocean_mask25.zarr')
M5 = xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/Ocean_mask05.zarr')

### Averaging
OM4p25_av = OM4p25['thetao'][:,0].mean(dim='time').load()
# OM4p5nep_av = OM4p5nep['tos'].mean(dim='time').load()
# OM4p25z_av = OM4p25z['tos'].mean(dim='time').load()
OM4p5_av = OM4p5['thetao'][:,0].mean(dim='time').load()
# OM4p5mle_av = OM4p5mle['tos'].mean(dim='time').load()

W25_av = W25['ptemp'][:,0].mean(axis=0).load()
W5_av = W5['ptemp'][:,0].mean(axis=0).load()

RuntimeError: error during blosc decompression: -1

In [7]:
%%time
OM4p25_av = OM4p25_av.rename({'yh':'latitude','xh':'longitude'})
OM4p25_av['latitude'], OM4p25_av['longitude'] = W25['latitude'], W25['longitude']

# OM4p5nep_av = OM4p5nep_av.rename({'yh':'latitude','xh':'longitude'})
# OM4p5nep_av['latitude'], OM4p5nep_av['longitude'] = W5_av['latitude'],W5_av['longitude']

# OM4p25z_av = OM4p25z_av.rename({'yh':'latitude','xh':'longitude'})
# OM4p25z_av['latitude'], OM4p25z_av['longitude'] = W25_av['latitude'],W25_av['longitude']

OM4p5_av = OM4p5_av.rename({'yh':'latitude','xh':'longitude'})
OM4p5_av['latitude'], OM4p5_av['longitude'] = W5_av['latitude'], W5_av['longitude']

# OM4p5mle_av = OM4p5mle_av.rename({'yh':'latitude','xh':'longitude'})
# OM4p5mle_av['latitude'], OM4p5mle_av['longitude'] = W5_av['latitude'],W5_av['longitude']

NameError: name 'OM4p25_av' is not defined

In [8]:
%%time
xq25 = G25 ['x'][:][::2,::2]
yq25 = G25 ['y'][:][::2,::2]
a25 = G25 ['area'][:]; a25 = a25[::2,::2]+a25[1::2,1::2]+a25[1::2,::2]+a25[1::2,::2]
m25 = M25 ['mask'][:]; a25 = a25*m25
xq5 = G5 ['x'][:][::2,::2]
yq5 = G5 ['y'][:][::2,::2]
a5 = G5 ['area'][:]; a5 = a5[::2,::2]+a5[1::2,1::2]+a5[1::2,::2]+a5[1::2,::2]
m5 = M5 ['mask'][:]; a5 = a5*m5

CPU times: user 9 ms, sys: 23 ms, total: 32 ms
Wall time: 12.7 ms


In [9]:
%%time
def stats(ax, area, anomaly, label):
    area = area.rename({'nx':'longitude','ny':'latitude'})
    
    mn = (anomaly*area).sum()/area.sum(); mn = mn.values
    sd = numpy.sqrt( ((anomaly-mn)**2*area).sum()/area.sum() ); sd = sd.values
    rms = numpy.sqrt( (anomaly**2*area).sum()/area.sum() ); rms = rms.values
    qmn, qmx = anomaly.min(), anomaly.max(); qmn, qmx = qmn.values, qmx.values
    print(label, 'mean =', mn, 'sd =', sd, 'rms =', rms, 'min =', qmn, 'max =', qmx )
    bb = ax.get_position()
    plt.gcf().text(bb.x0,bb.y1+.01,'mean=%.3f'%mn, horizontalalignment='left')
    plt.gcf().text(bb.x1,bb.y1+.01,'rms=%.3f'%rms, horizontalalignment='right')
#     plt.gcf().text(bb.x0,bb.y1+.01,'mean=%.3f$^\circ$C'%mn, horizontalalignment='left')
#     plt.gcf().text(bb.x1,bb.y1+.01,'rms=%.3f$^\circ$C'%rms, horizontalalignment='right')



CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.48 µs


In [10]:
%%time
fig = plt.figure(figsize=(12, 8))
vmin,vmax,ci,cmap = -2.25,2.25,.5,plt.cm.RdYlBu_r
axes = []
cilev = numpy.arange(vmin-ci,vmax+ci*2,ci)
norm = matplotlib.colors.BoundaryNorm(boundaries=cilev, ncolors=cmap.N)

q = OM4p25_av - W25_av
ax = fig.add_subplot(2,2,1,projection=cartopy.crs.Robinson(central_longitude=-155))
im = ax.pcolormesh(xq25, yq25, q,
              transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, vmin=vmin, vmax=vmax)
stats(ax, a25, q, 'OMp25'); 
ax.coastlines()
axes.append(ax)
plt.title('a) OM4p25')

# q = OM4p5nep_av - W5_av
# ax = fig.add_subplot(2,2,2,projection=cartopy.crs.Robinson(central_longitude=-155))
# im = ax.pcolormesh(xq5, yq5, q,
#               transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
# stats(ax, a5, q, 'OMp5nep')
# ax.coastlines()
# axes.append(ax)
# plt.title('b) OM4p5n')

# q = OM4p25z_av - W25_av
# ax = fig.add_subplot(2,2,3,projection=cartopy.crs.Robinson(central_longitude=-155))
# im = ax.pcolormesh(xq25, yq25, q,
#               transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
# stats(ax, a25, q, 'OMp25z')
# ax.coastlines()
# axes.append(ax)
# plt.title('c) OM4p25z')

# q = OM4p5mle_av - W5_av
# ax = fig.add_subplot(2,2,3,projection=cartopy.crs.Robinson(central_longitude=-155))
# im = ax.pcolormesh(xq5, yq5, q,
#               transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
# stats(ax, a5, q, 'OM4p5mle')
# ax.coastlines()
# axes.append(ax)
# plt.title('c) OM4p25e')

q = OM4p5_av - W5_av
ax = fig.add_subplot(2,2,2,projection=cartopy.crs.Robinson(central_longitude=-155))
im = ax.pcolormesh(xq5, yq5, q,
              transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, vmin=vmin, vmax=vmax)
stats(ax, a5, q, 'OMp5')
ax.coastlines()
axes.append(ax)
plt.title('d) OM4p5')

ax = plt.gcf().add_axes((.25,.5,.5,.03))
cb = matplotlib.colorbar.ColorbarBase(ax=ax, cmap=cmap, norm=norm, boundaries=cilev,
                                      orientation='horizontal', extend='both')
cb.ax.set_title('SST bias ($^\circ$C)')

plt.savefig('sst-bias.png')

NameError: name 'OM4p25_av' is not defined

<Figure size 864x576 with 0 Axes>